## Demo 1: Integer & binary optimisation

Solve the optimisation problem

$$
\begin{array}{lrll}
\max. &2x &  -\;\;\; 1000y  & \\
\\
\text{s.t.} &x & \leq 750y\\
& \;\;\;&x \in \mathbb{Z}_+ &\\
&\;\;\; &y \in \{0,1\}&
\end{array}
$$

In [20]:
#Part (a)
using JuMP, Cbc #modelling language and solver

d1 = Model(with_optimizer(Cbc.Optimizer,logLevel = 0)) #creates the model, select the solver  

@variable(d1, x >= 0, Int) # creates the non-negative integer variable x
@variable(d1, y, Bin) # creates the binary variable x

# @constraint(d1, x <= 750y) # constraint 1

@objective(d1, Max, 2x-1000y) # declare the objective function

# optimize!(d1) # solve the optimisation problem

# # Printing out the solution
# x_value = value.(x)
# y_value = value.(y)
# print("Optimal values: $(x_value),$(y_value),\nOptimal objective: $(objective_value(d1))\n")

Optimal values: 750.0,1.0,
Optimal objective: 500.0


In [21]:
#Part (b)

# comment out constraint one (and optimizie! and printing) above and add new constraint
@constraint(d1, x <= 200 + 550y)

# then run optimisation again
optimize!(d1)

#Printing out the solution
x_value = value.(x)
y_value = value.(y)
print("Optimal values: $(x_value),$(y_value),\nOptimal objective: $(objective_value(d1))\n")


Optimal values: 750.0,1.0,
Optimal objective: 500.0


In [22]:
#Part (c)

d1c = Model(with_optimizer(Cbc.Optimizer,logLevel = 0)) #creates the model, select the solver  
M = 10000

@variable(d1c, x[1:2] >= 0, Int) # creates the non-negative integer variable x
@variable(d1c, y, Bin) # creates the binary variable x

@constraint(d1c, x[1] + x[2] <= 15)
@constraint(d1c, x[1] - x[2] <= M*y)
@constraint(d1c, x[2] - x[1] <= M*(1-y))

@objective(d1c, Max, 25x[1]+32x[2]+450y+270*(1-y))

optimize!(d1c)

x_value = value.(x)
y_value = value.(y)
print("Optimal values: $(x_value),$(y_value),\nOptimal objective: $(objective_value(d1c))\n")

Optimal values: [8.0, 7.0],1.0,
Optimal objective: 874.0


## Problem 1: Sudoku

In [2]:
using JuMP, Cbc #modelling language and solver

P2 = Model(with_optimizer(Cbc.Optimizer,logLevel = 0)) #creates the model, select the solver  
I = 1:9
J = 1:9
K = 1:9
M = 1:3
N = 1:3

@variable(P2, x[i in I,j in J,k in K], Bin) # creates the binary variables x_ijk

@constraint(P2, [i in I, j in J], sum(x[i,j,k] for k in K) == 1) # can only have 1 value in each cell (i,j)
@constraint(P2, [j in J, k in K], sum(x[i,j,k] for i in I) == 1) # can only have one value in each row
@constraint(P2, [i in I, k in K], sum(x[i,j,k] for j in J) == 1) # can only have one value in each coulmn
@constraint(P2, [k in K, m in N, n in N], sum(sum([x[i,j,k] for j in (3*n-2):(3*n)]) for i in (3*m-2):(3*m)) == 1) # only one value found is each subgrid

@constraint(P2, x[1,2,6]+x[1,4,1]+x[1,6,4]+x[1,8,5] == 4)
@constraint(P2, x[2,3,8]+x[2,4,3]+x[2,6,5]+x[2,7,6] == 4)
@constraint(P2, x[3,1,2]+x[3,7,7] == 2)
@constraint(P2, x[4,1,8]+x[4,4,4]+x[4,6,7]+x[4,9,6] == 4)
@constraint(P2, x[5,3,6]+x[5,7,3] == 2)
@constraint(P2, x[6,1,7]+x[6,4,9]+x[6,6,1]+x[6,9,4] == 4)
@constraint(P2, x[7,1,5]+x[7,9,2] == 2)
@constraint(P2, x[8,3,7]+x[8,4,2]+x[8,6,6]+x[8,7,9] == 4)
@constraint(P2, x[9,2,4]+x[9,4,5]+x[9,6,8]+x[9,8,7] == 4)


@objective(P2, Min,100) # declare the dummy objective function

optimize!(P2) # solve the optimisation problem

#Printing out the solution

# this prints out all the arrays for each value
x_value = value.(x)
# print("Optimal values: $(x_value)")

# this creates the sudoku
function print_sudoku(x_value)
    sudoku = zeros(9,9) 
    for i in I
        sudoku .= sudoku .+ x_value[:,:,i]*i
    end    
    return sudoku
end

sudoku = print_sudoku(x_value)



9×9 Array{Float64,2}:
 9.0  6.0  3.0  1.0  7.0  4.0  2.0  5.0  8.0
 1.0  7.0  8.0  3.0  2.0  5.0  6.0  4.0  9.0
 2.0  5.0  4.0  6.0  8.0  9.0  7.0  3.0  1.0
 8.0  2.0  1.0  4.0  3.0  7.0  5.0  9.0  6.0
 4.0  9.0  6.0  8.0  5.0  2.0  3.0  1.0  7.0
 7.0  3.0  5.0  9.0  6.0  1.0  8.0  2.0  4.0
 5.0  8.0  9.0  7.0  1.0  3.0  4.0  6.0  2.0
 3.0  1.0  7.0  2.0  4.0  6.0  9.0  8.0  5.0
 6.0  4.0  2.0  5.0  9.0  8.0  1.0  7.0  3.0

## Problem 3: Store locations

In [9]:
using JuMP, Cbc

P3 = Model(with_optimizer(Cbc.Optimizer,logLevel = 0)) #creates the model, select the solver

I = 1:10
J = 1:10

A = [1 1 0 0 1 1 0 0 0 1;
    1 1 1 0 0 0 1 1 0 0;
    0 1 1 0 0 1 0 0 1 0;
    0 0 0 1 0 0 1 1 0 1;
    1 0 0 0 1 1 0 0 1 0;
    1 0 1 0 1 1 1 1 0 0;
    0 1 0 1 0 1 1 0 0 0;
    0 1 0 1 0 1 0 1 1 0;
    0 0 1 0 1 0 0 1 1 1;
    1 0 0 1 0 0 0 0 1 1]

p = [10000 15000 28000 30000 40000 30000 20000 15000 60000 12000]

@variable(P3, x[j in J], Bin) # creates the binary variables x_j

@constraint(P3, [i in I], sum(A[i,j] * x[j] for j in J if j != i) >= 1)

@objective(P3, Min, sum((1/p[j]) * x[j] for j in J))

optimize!(P3) # solve the optimisation problem
x_value = value.(x)
print("Optimal values: $(x_value)")


Optimal values: 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 1.0
 1.0
 0.0

## Problem 4: IP and LP

The LP is:
    \begin{align*}
        \max. &z = 3x_1 + 4x_2 + 5x_3\\
        \text{s.t. }& x_1 + 0.25 x_2 + 0.5 x_3 \leq 3\\
        &x_1 \geq 1\\
        &1 \leq x_2 \leq 2\\
        &x_3 \geq 1
    \end{align*}
    
The IP adds the domain that the decision variables must be integer, $x_1,\ x_2, \ x_3 \in \mathbb Z_+$.

In [4]:
using JuMP, Cbc #modelling language and solver

LP = Model(with_optimizer(Cbc.Optimizer,logLevel = 0)) #creates the model, select the solver  

@variable(LP, x[1:3] >= 0)

@constraint(LP, x[1] + 0.25x[2]+0.5x[3] <= 3) 
@constraint(LP,  x[1] >= 1)
@constraint(LP, 1 <= x[2] <= 2)
@constraint(LP, x[3] >= 1)

@objective(LP, Max, 3x[1]+4x[2]+5x[3]) 

optimize!(LP) 

# Printing out the solution
x_value = value.(x)
print("Optimal values for LP: $(x_value),\nOptimal objective: $(objective_value(LP))\n")

Optimal values for LP: [1.0, 2.0, 3.0],
Optimal objective: 26.0


In [6]:
using JuMP, Cbc #modelling language and solver

IP = Model(with_optimizer(Cbc.Optimizer,logLevel = 0)) #creates the model, select the solver  

@variable(IP, x[1:3] >= 0, Int)

@constraint(IP, x[1] + 0.25x[2]+0.5x[3] <= 3) 
@constraint(IP,  x[1] >= 1)
@constraint(IP, 1 <= x[2] <= 2)
@constraint(IP, x[3] >= 1)

@objective(IP, Max, 3x[1]+4x[2]+5x[3]) 

optimize!(IP) 

# Printing out the solution
x_value = value.(x)
print("Optimal values for IP: $(x_value),\nOptimal objective: $(objective_value(IP))\n")

Optimal values for IP: [1.0, 2.0, 3.0],
Optimal objective: 26.0
